In [20]:
'''
Delta-beta calculation script

Purpose: this script takes delta-beta output and calculates the 'combined impact' by adding together the three terms from this equation:

fulladapt_impact = HRshareFA*beta^H*(T_2099 - T_2015) + 
                 (1 - HRshareFA)*beta^L*(T_2099 - T_2015) + 
                 (HRshareFA- HRshareIA)*(beta^H - beta^L)*T_2015
                 
incadapt_impact = HRshareIA*beta^H*(T_2099 - T_2015) + 
                 (1 - HRshareIA)*beta^L*(T_2099 - T_2015)
                 
noadapt_impact = HRshareNA*beta^H*(T_2099 - T_2015) + 
                 (1 - HRshareNA)*beta^L*(T_2099 - T_2015) + 
                 (HRshareNA- HRshareIA)*(beta^H - beta^L)*T_2015

Parameters:

@model     :  the name of the model (and thus the name of folder that output is stored in)
@regions   :  a list of regions for which delta-beta CSVs have been outputted. They are generated from
                repos/gcp-labor/3_projection/deltabetas/yellow_purple_script_labor.R

'''
__author__ = 'Kit Schwarz'
__contact__ = 'csschwarz@uchicago.edu'
__version__ = '1.0'

############
# LIBRARIES
############

import xarray as xr
import pandas as pd
import getpass

############
# PARAMETERS
############

model = 'uninteracted_main_model'
# select: uninteracted_main_model_w_chn, uninteracted_main_model

regions = ['SDN.4.11.49.163', 'THA.3.R3edeff05b7928bfc', 'USA.5.221', 'CAN.3.50.1276'] 

In [21]:


############
# PATHWAYS
############

username = getpass.getuser()
db_root = '/mnt/CIL_labor/3_projection/deltabetas/spline_27_37_39'

if model == 'uninteracted_main_model_w_chn':
    
    proj_root = '/shares/gcp/outputs/labor/impacts-woodwork/uninteracted_main_model_w_chn/uninteracted_splines_w_chn_21_37_41_by_risk_empshare_noFE_YearlyAverageDay/rcp85/CCSM4/high/SSP3'
    rf = f'/home/{username}/repos/labor-code-release-2020/output/rf/uninteracted_reg_w_chn/uninteracted_reg_w_chn_full_response.csv'
    
elif model == 'uninteracted_main_model':
    
#     proj_root = '/shares/gcp/outputs/labor/impacts-woodwork/test_rcc/uninteracted_splines_27_37_39_by_risk_empshare_noFE_YearlyAverageDay/rcp85/CCSM4/high/SSP3'
#     rf = f'/home/{username}/repos/labor-code-release-2020/output/rf/uninteracted_reg_comlohi/uninteracted_reg_comlohi_full_response.csv'

    proj_root = '/shares/gcp/outputs/labor/impacts-woodwork/point_estimate_google_rebased/median/rcp85/CCSM4/high/SSP3'
    
else:
    
    print("Your model is unrecognized.")

In [22]:
############
# GET BETAS
############

high = [pd.read_csv(f'{db_root}/{model}/deltabeta_high{region}-2099.csv') for region in regions]
low = [pd.read_csv(f'{db_root}/{model}/deltabeta_low{region}-2099.csv')  for region in regions]

temp_bins = [df[['bin', 'T[2015]', 'T[diff]']] for df in high]

# subset columns and drop useless rows
subset_cols = ['bin', 'beta^F', 'beta^F*T[2099]-beta^I*T[2015]']
high = [df.loc[~df.bin.isin(['Total <20C', 'Total >20C', 'Total'])][subset_cols] for df in high]
low = [df.loc[~df.bin.isin(['Total <20C', 'Total >20C', 'Total'])][subset_cols] for df in low]

###################
# FUNCTION: get_lrt
###################

def get_climtas(regions, model):
    
    df = xr.open_dataset(f"/shares/gcp/outputs/temps/rcp85/{model}/climtas.nc4").to_dataframe().reset_index()
    climtas = [df.loc[(df.year == 2099) & (df.regions == region)]['averaged'].iloc[0] for region in regions]
    
    return climtas

########################
# FUNCTION: get_loggdppc
########################

def get_loggdppc(regions):
    
    df = pd.read_csv("/shares/gcp/outputs/labor/impacts-woodwork/test_rcc_copy/" +
        "uninteracted_splines_27_37_39_by_risk_empshare_noFE_YearlyAverageDay/rcp85/CCSM4/high/SSP3/" +
        "test_rcc_main_model_single_config-allcalcs-uninteracted_main_model.csv",
        skiprows=26, usecols=['region', 'year', 'climtas', 'loggdppc'])
    
    loggdppc = [df.loc[(df.year == 2099) & (df.region == region)]['loggdppc'].iloc[0] for region in regions]
    
    return loggdppc

############
# FUNCTION: get_shares
############
    
def get_shares(regions, adapt_list = ['fulladapt', 'incadapt', 'noadapt'], proj_root=proj_root, model=model):
    
    dict = {}
    
    for adapt in adapt_list:
        
        if adapt == "fulladapt":
            df = xr.open_dataset(f'{proj_root}/{model}.nc4').to_dataframe().reset_index()
        else:
            df = xr.open_dataset(f'{proj_root}/{model}-{adapt}.nc4').to_dataframe().reset_index()

        dict[f'{adapt}'] = [df.loc[(df.year == 2099) & (df.regions == region)]['clip'].iloc[0] for region in regions]
    
    return dict

############################
# FUNCTION: calc_delta_beta
############################

def calc_delta_beta(temp_bins, high, low, f_share, i_share, n_share):
    '''
    Calculates the fulladapt, incadapt, and noadapt impact for a given region.
    Input: a set of pd.DataFrames() containing the high and low betas, the temperature bins and days,
            and the high-risk share under fulladapt and incadapt scenarios.
            
    Output: a single pd.DataFrame().
    '''
    
    # rename columns according to risk group
    high.columns = [col + '_high' for col in high.columns]
    low.columns = [col + '_low' for col in low.columns]
    
    # merge together
    df = (temp_bins
                   .merge(high,
                          left_on='bin', right_on='bin_high')
                   .merge(low,
                          left_on='bin', right_on='bin_low')
              )
    
    # add in the incadapt and fulladapt shares
    df['fulladapt_share'] = f_share
    df['incadapt_share'] = i_share
    df['noadapt_share'] = n_share
    
    # calculating terms -> we do this for all the ref temps selected,
    # plus the basic delta beta with reference temperature 27C
    
    for adapt in 'fulladapt', 'incadapt', 'noadapt' :
        
        df[f'high_risk_term_{adapt}'] = df[f'{adapt}_share']      * df[f'beta^F_high'] * df['T[diff]']
        df[f'low_risk_term_{adapt}']  = (1- df[f'{adapt}_share']) * df[f'beta^F_low']  * df['T[diff]']
        df[f'extra_term_{adapt}']     = (df[f'{adapt}_share'] - df['incadapt_share']) * (df[f'beta^F_high'] - df[f'beta^F_low']) * df['T[2015]']
        df[f'comb_response_{adapt}']  = df[f'high_risk_term_{adapt}'] + df[f'low_risk_term_{adapt}'] + df[f'extra_term_{adapt}']
    
    return df

############################
# FUNCTION: get_daily_minutes
############################

def get_daily_minutes(df, cols):
    
    '''
    Calculates the daily average of a set of columns passed to cols.
    Input: a single pd.DataFrame(), and a list of cols.
            
    Output: a list of averages.
    '''
    
    numbers = [df[col].sum()/365 for col in cols]
    
    return numbers

In [23]:
############################
# GET RESULTS
############################

results = pd.DataFrame(index=regions)

# get shares
shares = get_shares(regions=regions)

for key, value in zip(shares.keys(), shares.values()):
    results[f'{key}_Hshare'] = value
    

# get loggdppc
results['loggdppc'] = get_loggdppc(regions)

# get climtas
for climmodel in ['CCSM4', 'surrogate_GFDL-CM3_99']:
    results[f'climtas_{climmodel}'] = get_climtas(regions, model=climmodel)
    
# delta beta results
delta_beta = [calc_delta_beta(a,b,c,d,e,f) for a,b,c,d,e,f in zip(
    temp_bins, high, low, shares.get('fulladapt'), shares.get('incadapt'), shares.get('noadapt'))]

# convert certain columns to daily
daily_cols = ['comb_response_fulladapt', 'comb_response_incadapt', 'comb_response_noadapt']
results_daily = [get_daily_minutes(df, daily_cols) for df in delta_beta]
daily_minutes = pd.DataFrame(results_daily, columns = daily_cols, index = regions)

results = results.merge(daily_minutes,
                       left_index=True,
                       right_index=True)



In [13]:
# export this baby
results.T

,SDN.4.11.49.163,THA.3.R3edeff05b7928bfc,USA.5.221,CAN.3.50.1276
fulladapt_Hshare,0.710272,0.332272,0.046444,0.100510
incadapt_Hshare,0.511133,0.214898,0.046444,0.046444
noadapt_Hshare,0.769325,0.457875,0.142215,0.183243
loggdppc,9.527578,10.596125,11.437371,11.502124
climtas_CCSM4,34.952217,32.420759,17.957219,8.891408
climtas_surrogate_GFDL-CM3_99,40.674934,37.254295,22.084608,15.452497
comb_response_fulladapt,-19.399385,-7.585463,0.188929,-0.416269
comb_response_incadapt,-12.102032,-5.695679,0.188929,0.062452
comb_response_noadapt,-21.563363,-9.607743,-0.033610,-1.148821


In [57]:
df = xr.open_dataset(f'{proj_root}/{model}-noadapt.nc4').to_dataframe().reset_index()
aggregated = xr.open_dataset(f'{proj_root}/{model}-noadapt-pop-aggregated.nc4').to_dataframe().reset_index()
levels = xr.open_dataset(f'{proj_root}/{model}-noadapt-pop-levels.nc4').to_dataframe().reset_index()

In [27]:
regions

['SDN.4.11.49.163', 'THA.3.R3edeff05b7928bfc', 'USA.5.221', 'CAN.3.50.1276']

In [55]:
# ok so we can see clip doesn't change for any regions

min_clip = df.groupby('regions')['clip'].min()
max_clip = df.groupby('regions')['clip'].max()

new = min_clip - max_clip
new.loc[new != 0]

Series([], Name: clip, dtype: float32)

In [51]:
# in pop-aggregated version, clip changes

aggregated.loc[aggregated.regions == '']

,region,year,regions,rebased,lowriskimpacts,highriskimpacts,clip,rebased_new
0,0,1981,,0.184523,0.702045,14.677774,0.531715,0.184523
1,0,1982,,0.418847,0.765806,15.035634,0.531715,0.418847
2,0,1983,,0.092191,0.699782,14.515599,0.531715,0.092191
3,0,1984,,0.141765,0.703546,14.610488,0.531715,0.141765
4,0,1985,,0.198151,0.724498,14.670691,0.531715,0.198151
...,...,...,...,...,...,...,...,...
115,0,2096,,-4.668433,-0.837564,8.621712,0.612654,-4.668433
116,0,2097,,-3.679210,-0.583626,9.984797,0.612654,-3.679210
117,0,2098,,-4.628629,-0.818385,8.708600,0.612654,-4.628629
118,0,2099,,-4.628160,-0.814034,8.732748,0.612654,-4.628160


In [56]:
min_clip = aggregated.groupby('regions')['clip'].min()
max_clip = aggregated.groupby('regions')['clip'].max()

new = min_clip - max_clip
new.loc[new != 0]

regions
               -0.083270
ATA                  NaN
ATF                  NaN
AUS.1                NaN
AUS.3                NaN
BRA.8.836            NaN
BRA.8.837            NaN
BVT                  NaN
CHN.21               NaN
CHN.21.226           NaN
CL-                  NaN
ESP.6.27.191         NaN
ESP.6.27.192         NaN
FUND-#N/A            NaN
FUND-ANZ       -0.002663
FUND-CAM       -0.014897
FUND-CHI       -0.001227
FUND-EEU       -0.015000
FUND-FSU       -0.016475
FUND-JPK       -0.001224
FUND-LAM       -0.008282
FUND-MAF       -0.001332
FUND-MDE       -0.045469
FUND-SAS       -0.012083
FUND-SEA       -0.016057
FUND-SIS       -0.030013
FUND-SSA       -0.019627
FUND-USA       -0.000236
FUND-WEU       -0.001303
HMD                  NaN
IOT                  NaN
NZL.10.42            NaN
SGS                  NaN
SP-                  NaN
TWN.2                NaN
ZAF.9                NaN
Name: clip, dtype: float32

In [58]:
levels

,region,year,regions,rebased,lowriskimpacts,highriskimpacts,clip,rebased_new
0,0,1981,CAN.1.2.28,1.156390e+03,5172.064941,75483.468750,2283.663818,1.156390e+03
1,0,1982,CAN.1.2.28,-4.066874e+02,4761.744141,69180.609375,2283.663818,-4.066880e+02
2,0,1983,CAN.1.2.28,-2.775326e+03,4111.332520,59747.156250,2283.663818,-2.775327e+03
3,0,1984,CAN.1.2.28,1.091810e+03,5169.767578,75162.804688,2283.663818,1.091810e+03
4,0,1985,CAN.1.2.28,6.635757e+02,4999.570801,73673.593750,2283.663818,6.635753e+02
...,...,...,...,...,...,...,...,...
2925355,24377,2096,BWA.4.13,-7.556717e+05,-206667.000000,710122.312500,45072.800781,-7.556717e+05
2925356,24377,2097,BWA.4.13,-1.070408e+06,-330570.468750,41869.105469,45072.800781,-1.070408e+06
2925357,24377,2098,BWA.4.13,-7.494238e+05,-203541.734375,722154.625000,45072.800781,-7.494238e+05
2925358,24377,2099,BWA.4.13,-8.865341e+05,-259081.296875,433934.781250,45072.800781,-8.865342e+05


In [54]:
new

regions
         -0.08327
ABW       0.00000
AFG       0.00000
AFG.1     0.00000
AFG.10    0.00000
           ...   
ZWE.5     0.00000
ZWE.6     0.00000
ZWE.7     0.00000
ZWE.8     0.00000
ZWE.9     0.00000
Name: clip, Length: 5716, dtype: float32